In [4]:
import pandas as pd
import numpy as np
import os

In [9]:
def create_dataframe(ficheros_list):
    '''
    Params:
        fichero list
    '''
    global df_list
    if isinstance(ficheros_list, list):
        try:
            df_list = list()
            for idx, f in enumerate(ficheros_list):
                df_temp = pd.read_csv(os.path.join('data',f))
                df_list.append(df_temp)
        except Exception as e:
            print(e)    
    else:
        "Solo admite ficheros en lista"

In [7]:
lista_ficheros = os.listdir("data")
create_dataframe(lista_ficheros)

In [17]:
import sqlite3

In [32]:
list() df_list[1].values

array([100896, 4, '05/15/2023', 50, 100275, 33, '01/21/2023', 5, 100919,
       34, '01/20/2023', 50, 100983, 82, '04/11/2023', 20, 100537, 96,
       '05/01/2023', 15, 100279, 98, '03/04/2023', 30, 100121, 104,
       '01/18/2023', 15, 100971, 109, '02/18/2023', 5, 100623, 117,
       '03/28/2023', 50, 100262, 120, '04/07/2023', 40, 100071, 129,
       '02/28/2023', 30, 100561, 133, '01/05/2023', 30, 100741, 177,
       '04/13/2023', 20, 100658, 187, '04/15/2023', 30, 100563, 188,
       '05/04/2023', 50, 100603, 205, '01/05/2023', 15, 100236, 210,
       '05/24/2023', 10, 100464, 215, '01/19/2023', 20, 100106, 219,
       '02/02/2023', 30, 100830, 231, '03/08/2023', 10, 100540, 233,
       '02/19/2023', 5, 100201, 247, '04/25/2023', 20, 100354, 249,
       '03/04/2023', 10, 100426, 253, '04/17/2023', 5, 100903, 285,
       '05/22/2023', 15, 100203, 290, '02/05/2023', 15, 100877, 331,
       '02/08/2023', 40, 100049, 401, '04/02/2023', 30, 100821, 409,
       '05/27/2023', 30, 100847,

In [39]:
df_list[0].dtypes[2]

dtype('O')

In [78]:
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INT',
    'int32': 'INT',
    'float64': 'REAL',
    'float32': 'REAL',
    'bool': 'INTEGER',
    'datetime64[ns]': 'NUMERIC',
    'date': 'NUMERIC',
    'time': 'NUMERIC',
    'str' : 'BLOB'
}

In [42]:
df_list[0].dtypes

Unnamed: 0          int64
customer_id         int64
birthday           object
country            object
spanish_speaker      bool
join_date          object
tier               object
dtype: object

In [44]:
df_list[0].columns

Index(['Unnamed: 0', 'customer_id', 'birthday', 'country', 'spanish_speaker',
       'join_date', 'tier'],
      dtype='object')

In [93]:
def insert_sql_table(df_in_list, db_name=None, db_table_name=None):
    '''
    INSERT SQL TABLE
    '''
    if not db_name is None:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
           
        if isinstance(df_in_list, list):
            try:
                for dfl in df_in_list:
                    # Generate values
                    column_names = dfl.columns.tolist()
                    data_types = dfl.dtypes.tolist()
                    values = dfl.values.tolist()

                    for table in db_table_name:
                        table_name = table.split('-')[0].strip()
                        # Generate the CREATE TABLE statement
                        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
                        for column, dtype in zip(column_names, data_types):
                            sql_dtype = dtype_mapping.get(str(dtype), 'VARCHAR')
                            create_table_query += f"{column} {sql_dtype}, "
                        create_table_query = create_table_query.rstrip(", ") + ")"
                        
                        cursor.execute(create_table_query)
                        # Guardamos los cambios haciendo un commit
                        conn.commit()
                        
                        insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES "
                        insert_values = []
                        for row in values:
                            row_values = [f"'{value}'" if isinstance(value, str) else str(value) for value in row]
                            insert_values.append("(" + ", ".join(row_values) + ")")
                        insert_query += ", ".join(insert_values)

                        # Execute the INSERT INTO statement
                        cursor.execute(insert_query)
                        
                        # Guardamos los cambios haciendo un commit
                        conn.commit()
            except Exception as e:
                print(e)
            
            finally:
                conn.close()
        else:
            "Se requiere un fichero en lista"

In [94]:
insert_sql_table(df_list, db_name='carrefive.db', db_table_name=lista_ficheros)

unrecognized token: ":"


In [95]:
# Realizamos la conexión

conn = sqlite3.connect("carrefive.db")

tables = pd.read_sql(""" select *
                        from sqlite_master 
                        where type='table'
                        order by name;
                        """, conn)
tables

,type,name,tbl_name,rootpage,sql
